### Searching for Lyapunov Candidate!

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (
    Jacobian,
    MathematicalProgram,
    Solve,
    SymbolicVectorSystem,
    ToLatex,
    Variable,
    Variables,
)
from pydrake.symbolic import Polynomial

### Define equations for Dynamic Bicycle model

In [67]:
# declare variables
from sympy import *
from IPython.display import display, Markdown, Latex

m, j_z, c_af, c_s, c_d = symbols("m J_z C_{\\alpha_f} C_{\\Sigma} C_{\\Delta}")
x_dot, y_dot, psi_dot, delta, y_ddot, psi_ddot = symbols("\\dot{x} \\dot{y} \\dot{\\psi} \\delta \\ddot{y} \\ddot{\\psi}")

# length is constant
l = 1

v_dot = Matrix([
[-c_s/(m*x_dot) * y_dot - c_d * l/(m*x_dot) * psi_dot + c_af/m * delta - x_dot * psi_dot],
[-c_d/(j_z * x_dot) * y_dot - c_s * l**2/(j_z * x_dot) * psi_dot + c_af * l/j_z * delta]]
)

# define inertial matrix
M = Matrix([[m, 0], [0, j_z]])

# get LHS (LHS = M * v_dot)
LHS = simplify(M * Matrix([[y_ddot], [psi_ddot]]))

#print LHS in latex
display(Markdown("$$" + latex(LHS) + "$$"))

# get RHS (RHS = M * v_dot)
RHS = simplify(M * v_dot)
display(Markdown("$$" + latex(RHS) + "$$"))


# make the W regressor
theta = Array([m, j_z, c_af, c_s, c_d])

# take the jacobian of v_dot with respect to theta
W_v = (RHS).jacobian(theta)
display(Markdown("$$W_v = " + latex(W_v) + "$$"))

# make Big W regressor
W = (LHS - RHS).jacobian(theta)
display(Markdown("$$\mathbb{W} = " + latex(W) + "$$"))




$$\left[\begin{matrix}\ddot{y} m\\J_{z} \ddot{\psi}\end{matrix}\right]$$

$$\left[\begin{matrix}- \frac{C_{\Delta} \dot{\psi}}{\dot{x}} - \frac{C_{\Sigma} \dot{y}}{\dot{x}} + C_{\alpha_f} \delta - \dot{\psi} \dot{x} m\\\frac{- C_{\Delta} \dot{y} - C_{\Sigma} \dot{\psi} + C_{\alpha_f} \delta \dot{x}}{\dot{x}}\end{matrix}\right]$$

$$W_v = \left[\begin{matrix}- \dot{\psi} \dot{x} & 0 & \delta & - \frac{\dot{y}}{\dot{x}} & - \frac{\dot{\psi}}{\dot{x}}\\0 & 0 & \delta & - \frac{\dot{\psi}}{\dot{x}} & - \frac{\dot{y}}{\dot{x}}\end{matrix}\right]$$

$$\mathbb{W} = \left[\begin{matrix}\ddot{y} + \dot{\psi} \dot{x} & 0 & - \delta & \frac{\dot{y}}{\dot{x}} & \frac{\dot{\psi}}{\dot{x}}\\0 & \ddot{\psi} & - \delta & \frac{\dot{\psi}}{\dot{x}} & \frac{\dot{y}}{\dot{x}}\end{matrix}\right]$$

### Dynamic Bicycle Model

\begin{equation}
    \ddot{y} = -\frac{C_{\Sigma}}{m \dot{x}}\dot{y}
    -\frac{C_{\Delta}l}{m \dot{x}}\dot{\psi}
    + \frac{C_{\alpha f}}{m} \delta
    -\dot{x}\dot{\psi}
\end{equation}

\begin{equation}
    \ddot{\psi} = -\frac{C_{\Delta}l}{J_z \dot{x}} \dot{y}
    -\frac{C_{\Sigma} l^{2}}{J_z \dot{x}}\dot{\psi}
    + \frac{C_{\alpha f}l}{J_z} \delta.
\end{equation}



\begin{equation}
    \delta = \frac{J_z}{C_{\alpha f} l}( \frac{C_{\Delta}l}{J_z \dot{x}} \dot{y}
    +\frac{C_{\Sigma} l^{2}}{J_z \dot{x}}\dot{\psi})
\end{equation}

$\theta = [m \quad J_z \quad C_{\alpha f}  \quad C_{\sigma} \quad C_{\Delta}]  \qquad \in \mathbb{R}^5$

In [8]:
def sos_bicycle():
    prog = MathematicalProgram()

    # define known parameters 
    m = 1.0  # mass
    J_z = 0.1  # moment of inertia
    l = 0.5  # length
    C_af = 15 
    C_sigma = 35
    C_delta = -5
    x_dot = 1

    # define state variables
    x = prog.NewIndeterminates(2, "x")
    y_dot = x[0]
    psi_dot = x[1]

    # define input variables & dynamics
    delta_in = (J_z/(C_af *l) * (C_sigma* l/(J_z * x_dot) * y_dot + C_sigma * l**2/(J_z * x_dot) * psi_dot))
    y_ddot = -C_sigma/(m*x_dot) * y_dot - C_delta * l/(m*x_dot) * psi_dot + C_af/m * delta_in - x_dot * psi_dot
    psi_ddot = -C_delta/(J_z * x_dot) * y_dot - C_sigma * l**2/(J_z * x_dot) * psi_dot + C_af * l/J_z * delta_in

    # define f(x) for bicycle model
    f = [y_ddot, psi_ddot]

    # create a blank SOS polynomial
    V = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()
    print("Candidate:")
    display(Markdown("$V(x) = " + ToLatex(V) + "$"))
    # add constraints to scale the result
    prog.AddLinearConstraint(V.Substitute({x[0]: 0, x[1]: 0}) == 0)
    prog.AddLinearConstraint(V.Substitute({x[0]: 1, x[1]: 0}) == 1)
    # define Vdot
    Vdot = V.Jacobian(x).dot(f)
    # Make sure -Vdot is SOS
    prog.AddSosConstraint(-Vdot)

    result = Solve(prog)
    assert result.is_success()

    print("Solution:")
    display(
        Markdown(
            "$V(x) = "
            + ToLatex(
                Polynomial(result.GetSolution(V))
                .RemoveTermsWithSmallCoefficients(1e-5)
                .ToExpression(),
                6,
            )
            + "$"
        )
    )


sos_bicycle()

Candidate:


$V(x) = (S_{2,2} + 2x_{0} x_{1} S_{1,0} + 2x_{0} S_{2,1} + x_{0}^{2} S_{1,1} + 2x_{1} S_{2,0} + x_{1}^{2} S_{0,0})$

Solution:


$V(x) = ( - 0.581187x_{0} x_{1} + 1.000000x_{0}^{2} + 0.084445x_{1}^{2})$